
<p style="border:2px solid black"> </p>
<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.8cm"></p>
<center>
<font size="6"><b>Understanding Music Listening Habits</b></font>
<p style="margin-bottom:-0.1cm"></p>
<font size="6"><b>Using Large-scale Smartphone Data</b>  </font>

<p style="margin-bottom:0.5cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<p style="margin-bottom:1cm"></p>
<font size="5"><b>Overview</b></font>
<br>
<font size="3"><b></b></font>
</center>
<p style="margin-bottom:0.4cm"></p>
<p style="border:2px solid black"> </p>

    

# Initialization
<p style="border:2px solid black"> </p>


The initializaiton procedure is defined in the notebook [Initialization](initialization.ipynb).

In [13]:
%run initialization.ipynb

# First look at the data
<p style="border:2px solid black"> </p>


The analysis described in this section is performed in the notebook [Preliminary data analysis](preliminary_data_analysis.ipynb). Main findings:

1. The raw data has multilevel column index, which we convert to a flat index, resulting in following 18 columns and corresponding descriptions (as stated in the provided avro files): 

    1. `id`: Activity UUID
    1. `deleted_time`: If this activity has been deleted, this is the timestamp when it occurred
    1. `useruuid`: User UUID
    1. `start_time`: Start time
    1. `end_time:` End time
    1. `devices`: Devices used during this activity
    1. `devices_name`: Name of the device
    1. `devices_type`: Device type (PHONE, SMARTBAND, SMARTCAMERA, UNKNOWN, ...)
    1. `devices_id`: Unique identifier of the device
    1. `tracks`: -
    1. `tracks_start_time`: -
    1. `tracks_end_time`: -
    1. `tracks_artist`: -
    1. `tracks_album`: -
    1. `tracks_title`: -
    1. `tracks_uri`: -
    1. `tracks_player`: -
    1. `vtracks_id`: - 
1. The raw data has **627,101,296** rows, each corresponding to an "activity". 

1. The data two columns where data contains lists: `devices` and `tracks`. We explode these columns so that each row corresponds to a playback of one song on one device. This increases the number of rows in the database to **3,571,278,161** 

1. Activity UUID (column `id`):
    1. No empty cells
    1. **46,360,934** id's have at least one duplicate (this number amounts to 7.4 % all rows in the raw database).
    
1. Activity Deleted (column `deleted_time`):
    1. Around 8.5 % of all the activities have the "Deleted time" property set. What to do with them:
        1. Nothing
        1. Delete the property (column)
        1. Delete the rows where the property is set
    